<a href="https://colab.research.google.com/github/felzr/tax-data/blob/master/tax_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importar a biblioteca
import pandas as pd

In [ ]:
file_path = "ISS_LC116 (1) 1.xlsx"

In [ ]:
# Loop para carregar cada aba como um DataFrame separado
aliquotas = pd.read_excel(file_path,sheet_name='CNAELC116XISS_SP',dtype=str)

In [ ]:
aliquotas = aliquotas[aliquotas['Cidade'] != 'Brasília']
aliquotas['Cod'] = aliquotas['Cod'].apply(lambda x: x.lstrip('0'))
aliquotas['Cod'] = aliquotas['Cod'].str.lstrip()
print(aliquotas.head())

   Código da Aliquota  Aliquota LC116 UniProfissional  Descrição   \
0                1.01       5.0     Y               Y         NaN   
1                1.02       5.0     Y             NaN         NaN   
2                1.03       5.0     Y             NaN         NaN   
3                1.04       5.0     Y             NaN         NaN   
4                1.05       5.0     Y             NaN         NaN   

           Data Inicio             Data Fim     Cidade   Cod  
0  2024-01-01 00:00:00  9999-12-31 00:00:00  São Paulo  1.01  
1  2024-01-01 00:00:00  9999-12-31 00:00:00  São Paulo  1.02  
2  2024-01-01 00:00:00  9999-12-31 00:00:00  São Paulo  1.03  
3  2024-01-01 00:00:00  9999-12-31 00:00:00  São Paulo  1.04  
4  2024-01-01 00:00:00  9999-12-31 00:00:00  São Paulo  1.05  


In [ ]:
# Exibir todos os valores da coluna 'Código da Aliquota'
valor_desejado = '17.20'  # Substitua '12345' pelo valor que você está procurando

# Filtrar o DataFrame para encontrar a linha correspondente
linha_encontrada = aliquotas[aliquotas['Cod'] == valor_desejado]
print(linha_encontrada)

     Código da Aliquota  Aliquota LC116 UniProfissional  Descrição   \
97                 17.2       5.0     Y             NaN         NaN   
361                17.2       5.0     Y             NaN         NaN   
557                17.2       5.0     Y             NaN         NaN   
752                17.2       5.0     Y             NaN         NaN   

             Data Inicio             Data Fim          Cidade    Cod  
97   2024-01-01 00:00:00  9999-12-31 00:00:00       São Paulo  17.20  
361  2024-01-01 00:00:00  9999-12-31 00:00:00  Rio de Janeiro  17.20  
557  2014-04-01 00:00:00  9999-12-31 00:00:00  Belo Horizonte  17.20  
752           01/04/2024           31/12/9999        Brasília  17.20  


In [ ]:
count_registros = aliquotas['Cod'].count()
print(count_registros)
print(aliquotas['Cod'].tolist())

816
[1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 10.01, 10.02, 10.03, 10.04, 10.05, 10.06, 10.07, 10.08, 10.09, 10.1, 11.01, 11.02, 11.03, 11.04, 11.05, 12.01, 12.02, 12.03, 12.04, 12.05, 12.06, 12.07, 12.08, 12.09, 12.1, 12.11, 12.12, 12.13, 12.14, 12.15, 12.16, 12.17, 13.02, 13.03, 13.04, 13.05, 14.01, 14.02, 14.03, 14.04, 14.05, 14.06, 14.07, 14.08, 14.09, 14.1, 14.11, 14.12, 14.13, 14.14, 15.01, 15.02, 15.03, 15.04, 15.05, 15.06, 15.07, 15.08, 15.09, 15.1, 15.11, 15.12, 15.13, 15.14, 15.15, 15.16, 15.17, 15.18, 16.01, 16.02, 17.01, 17.02, 17.03, 17.04, 17.05, 17.06, 17.08, 17.09, 17.1, 17.11, 17.12, 17.13, 17.14, 17.15, 17.16, 17.17, 17.18, 17.19, 17.2, 17.21, 17.22, 17.23, 17.24, 17.25, 18.01, 19.01, 2.01, 20.01, 20.02, 20.03, 21.01, 22.01, 23.01, 24.01, 25.01, 25.02, 25.03, 25.04, 25.05, 26.01, 27.01, 28.01, 29.01, 3.02, 3.03, 3.04, 3.05, 30.01, 31.01, 32.01, 33.01, 34.01, 35.01, 36.01, 37.01, 38.01, 39.01, 4.01, 4.02, 4.03, 4.04, 4.05, 4.06, 4.07, 4.08, 4.09, 4.1, 4.11

In [ ]:
with open('inserts_iss.sql', 'w') as file:
    # Iterar sobre as linhas do dataset
    for index, row in aliquotas.iterrows():
        # Construir o comando INSERT
        insert_sql = f"""
        INSERT INTO ecg_taxass_iss (
            ecg_taxass_iss_id, ad_client_id, ad_org_id, created, createdby, ecg_taxass_iss_uu, isactive,
            updated, updatedby, description, ecg_taxass_cod, ecg_taxass_aliquota, ecg_taxass_valorfixo,
            c_city_id, ecg_taxass_datainicio, ecg_taxass_datafim, ecg_taxass_islc116,
            ecg_taxass_itemlei, isdefault, ecg_taxass_isissfixo
        ) VALUES (
            (SELECT nextval('ecg_taxass_iss_sq')), 0, 0, NOW(), 100, uuid_generate_v4(), 'Y',
            NOW(), 100, (select substring(name,0,255) from ecg_taxass_lc116 etl where value ='{row['Cod']}'), '{row['Cod']}', {row['Aliquota']}, null,
            (select c_city_id from c_city cc where  name = '{row['Cidade']}'),
            TO_DATE('01/04/2024', 'dd/MM/yyyy'), TO_DATE('31/12/9999', 'dd/MM/yyyy'), 'Y',
            null, 'N', 'N'
        );
        """
        # Escrever o comando no arquivo
        file.write(insert_sql)

print("Arquivo SQL gerado com sucesso!")

Arquivo SQL gerado com sucesso!


In [ ]:
sp = pd.read_excel(file_path,sheet_name='CNAELC116XISS_SP',dtype=str)
bh = pd.read_excel(file_path,sheet_name='CNAELC116XISS_BH',dtype=str)
df = pd.read_excel(file_path,sheet_name='CNAELC116XISS_DF',dtype=str)
# Filtrar as linhas onde 'nao_mexer_nessa_coluna' não é nulo ou vazio
rj = pd.read_excel(file_path,sheet_name='CNAELC116XISS_RJ',dtype=str)

In [ ]:
merged_dataset = pd.concat([sp, rj, df, bh], ignore_index=True)
merged_dataset = merged_dataset[merged_dataset['nome_cidade'].notna() & (merged_dataset['nome_cidade'] != '')]


In [ ]:
print(merged_dataset)
merged_dataset['cod_iss_aliquota'] = merged_dataset['cod_iss_aliquota'].apply(lambda x: x.lstrip('0'))
merged_dataset.to_excel('cidades.xlsx', index=False)

     lc116                                    descricao_lc116  \
0    01.01             Análise e desenvolvimento de sistemas.   
1    01.02                                       Programação.   
2    01.03               Processamento de dados e congêneres.   
3    01.04  Elaboração de programas de computadores, inclu...   
4    01.05  Licenciamento ou cessão de direito de uso de p...   
..     ...                                                ...   
797   8.01  Ensino regular pré–escolar, fundamental, médio...   
798   8.02  Instrução, treinamento, orientação pedagógica ...   
799   9.01  Hospedagem de qualquer natureza em hotéis, apa...   
800   9.02  Agenciamento, organização, promoção, intermedi...   
801   9.03                                  Guias de turismo.   

    cod_iss_aliquota     nome_cidade ISS devido no municipio  \
0              01.01       São Paulo                       X   
1              01.02       São Paulo                       X   
2              01.03       

In [ ]:
with open('inserts_lc116iss.sql', 'w') as file:
    # Iterar sobre as linhas do dataset
    for index, row in merged_dataset.iterrows():
        # Construir o comando INSERT
        insert_sql = f"""
        INSERT INTO ecg_taxass_lc116iss (
            ecg_taxass_lc116iss_id , ad_client_id, ad_org_id, created, createdby, ecg_taxass_lc116iss_uu , isactive,
            updated, updatedby, ecg_taxass_iss_id ,ecg_taxass_lc116_id
        ) VALUES (
            (SELECT nextval('ecg_taxass_lc116iss_sq')), 0, 0, NOW(), 100, uuid_generate_v4(), 'Y',
            NOW(), 100, (select ecg_taxass_iss_id from ecg_taxass_iss eti  where ecg_taxass_cod = '{row['cod_iss_aliquota']}' and c_city_id =(select c_city_id from c_city cc where  name = '{row['nome_cidade']}')),
            (select ecg_taxass_lc116_id from ecg_taxass_lc116 etc where lpad(value,5 ,'0') = lpad( '{row['lc116']}' , 5,'0'))
        );
        """
        # Escrever o comando no arquivo
        file.write(insert_sql)

print("Arquivo SQL gerado com sucesso!")

Arquivo SQL gerado com sucesso!
